## Import dependencies

In [1]:
import requests
import bs4 as bs
import re
import pandas as pd

## Define State Dictionary for iteration

In [5]:
state_list= ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia",
"Hawaii" ,"Idaho" ,"Illinois" ,"Indiana" ,"Iowa" ,"Kansas" ,"Kentucky" ,"Louisiana" ,"Maine" ,"Maryland" ,"Massachusetts", 
"Michigan" ,"Minnesota" ,"Mississippi" ,"Missouri" ,"Montana" ,"Nebraska" ,"Nevada" ,"New Hampshire" ,"New Jersey" ,
"New Mexico" ,"New York" ,"North Carolina" ,"North Dakota" ,"Ohio" ,"Oklahoma" ,"Oregon" ,"Pennsylvania" ,"Rhode Island" ,
"South Carolina" ,"South Dakota" ,"Tennessee" ,"Texas" ,"Utah" ,"Vermont" ,"Virginia" ,"Washington" ,"West Virginia" ,
"Wisconsin" ,"Wyoming"]

state_list_test= ["Alabama","Alaska","North Carolina","South Dakota","Colorado"]

## Iterate the request for each state

In [6]:
# for state in state_dict.keys():
#     response= requests.post('https://www.brewersassociation.org/wp-admin/admin-ajax.php',
#                  data= {
#                      "action": "get_breweries",
#                     "_id": state,
#                     "search_by": "statename"
#                  })
#     state_dict[state]= response

response_list= []

for state in state_list:
    response= requests.post('https://www.brewersassociation.org/wp-admin/admin-ajax.php',
                 data= {
                     "action": "get_breweries",
                    "_id": state,
                    "search_by": "statename"
                 })
    if response.status_code == 200:
        response_list.append(response)
        print(f"{state} found!")
    else:
        print(f"Not found. Skipping {state}...")

print("/\\"*30)
print("REQUEST COMPLETE")

Alabama found!
Alaska found!
Arizona found!
Arkansas found!
California found!
Colorado found!
Connecticut found!
Delaware found!
Florida found!
Georgia found!
Hawaii found!
Idaho found!
Illinois found!
Indiana found!
Iowa found!
Kansas found!
Kentucky found!
Louisiana found!
Maine found!
Maryland found!
Massachusetts found!
Michigan found!
Minnesota found!
Mississippi found!
Missouri found!
Montana found!
Nebraska found!
Nevada found!
New Hampshire found!
New Jersey found!
New Mexico found!
New York found!
North Carolina found!
North Dakota found!
Ohio found!
Oklahoma found!
Oregon found!
Pennsylvania found!
Rhode Island found!
South Carolina found!
South Dakota found!
Tennessee found!
Texas found!
Utah found!
Vermont found!
Virginia found!
Washington found!
West Virginia found!
Wisconsin found!
Wyoming found!
/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
REQUEST COMPLETE


## Convert to BeautifulSoup object and prettify for easier reading

In [7]:
soup_list= [bs.BeautifulSoup(i.text,'html.parser') for i in response_list]

## Locate all the brewery classes

In [8]:
breweries= [soup.find_all(attrs= {"class":"brewery"}) for soup in soup_list]

## Convert to DataFrame

Having issues with this step right now because I made lists of lists... like a chump...

In [9]:
p= re.compile("<li>(.*?), ([A-Z][A-Z]) (\d{5})?")

Brewery_list= []
Address_list= []
City_list= []
State_list= []
ZIP_Code_list= []

fail_list= []

for state in breweries:
    for i in state:
#Since some breweries lack a ZIP Code, it throws an error.
        try:
            city_state_zip= i.find_all('li')[2]
#ZIP_Code_list should be produced first to throw the error before populating the other fields.
#This ensures all lists are the same length and can easily be added to the state_df.
            ZIP_Code_list.append(re.search(p, str(city_state_zip)).group(3))
            Brewery_list.append(i.find_all(attrs={"class":"name"})[0].string)
            Address_list.append(i.find_all(attrs={"class":"address"})[0].string.rstrip())
            City_list.append(re.search(p, str(city_state_zip)).group(1))
            State_list.append(re.search(p, str(city_state_zip)).group(2))
        except AttributeError:
            fail_list.append(i.find_all(attrs={"class":"name"})[0].string)

In [10]:
state_df= pd.DataFrame(
    {
            "Brewery": Brewery_list,
            "Address": Address_list,
            "City": City_list,
            "State": State_list,
            "ZIP Code": ZIP_Code_list
    })

In [11]:
state_df.head()

,Brewery,Address,City,State,ZIP Code
0,5 Rivers Brewing LLC,,Spanish Fort,AL,36527
1,Avondale Brewing Co,201 41st St S,Birmingham,AL,35222
2,Back Forty Beer Co,200 N 6th St,Gadsden,AL,35901
3,Back Forty Beer Company - Birmingham,3201 1st Avenue N,Birmingham,AL,35222
4,Below the Radar Brewing Co,220 Holmes Ave NE,Huntsville,AL,35801


In [12]:
state_df.describe()

,Brewery,Address,City,State,ZIP Code
count,8501,8501,8501,8501,8427
unique,8338,7452,3154,50,4958
top,Ballast Point Brewing Company,,Portland,CA,92121
freq,6,981,100,961,18


In [13]:
print(len(fail_list))
fail_list

8


['Cerveza Aldarra',
 'Pirate Republic Brewing',
 'Mount Vernon Brewery',
 'Cheeky Monkey Brewery and Cidery',
 'Cowaramup Brewing Company',
 'Eagle Bay Brewing Co',
 'Feral Brewing Company',
 'Northbridge Brewing Co']

In [14]:
state_df.to_csv("Resources/Brewers_Association_Data.csv")

In [16]:
state_df.drop_duplicates("Brewery")

,Brewery,Address,City,State,ZIP Code
0,5 Rivers Brewing LLC,,Spanish Fort,AL,36527
1,Avondale Brewing Co,201 41st St S,Birmingham,AL,35222
2,Back Forty Beer Co,200 N 6th St,Gadsden,AL,35901
3,Back Forty Beer Company - Birmingham,3201 1st Avenue N,Birmingham,AL,35222
4,Below the Radar Brewing Co,220 Holmes Ave NE,Huntsville,AL,35801
5,Big Beach Brewing Company,300 E 24th Ave,Gulf Shores,AL,36542
6,Birmingham District Brewing,2201 2nd Ave S Ste 102,Birmingham,AL,35233
7,Black Warrior Brewing Co.,2216 University Blvd,Tuscaloosa,AL,35401
8,Blue Pants Brewery,"500 Lanier Rd, Bld 1 - Ste A",Madison,AL,35758
9,Braided River Brewing Company,,Mobile,AL,36606


In [28]:
address_dups= state_df.loc[state_df.duplicated("Address")]
address_dups.to_csv("Resources/Duplicate_Address.csv")

In [31]:
address_dups_list= ['1 Main St',
'1000 N Frontage Rd Ste C',
'101 N Main St',
'102 S State St',
'106 S Main St',
'109 Main St',
'109 W Main St',
'111 N Main St',
'1128 N 9th St',
'118 Main St W',
'1195 Evans Ave',
'1225 S Gilbert St',
'124 Main St',
'140 E Park Ave',
'141 E Main St',
'14209 29th St E Ste 102',
'1480 Industrial Dr Ste C',
'150 W 96th St',
'15103 Surveyor Blvd',
'154 S Madison St',
'1627 S 7th St',
'1701 Telegraph Ave',
'18421 Gothard St Ste 100',
'19 N Main St',
'20 Main St',
'202 E Main St',
'21 E Main St',
'213 N Main St',
'2181 Logan Ave',
'220 W Main St',
'2236 1st Capitol Dr',
'2255 Falcon Ave',
'229 W 4th St',
'2315 Main St',
'236 Main St',
'25 Drexel Dr',
'25 Pine St',
'2600 W. 2nd Ave Unit 8',
'2607 W 17th St',
'300 W Main St',
'303 Main St',
'317 Springtown Rd',
'320 Pierce St',
'33 35th St Unit 6A',
'3900 Frontage Rd S',
'3900 Frontage Rd S',
'4 Main St',
'401 Main St',
'408 Shelden Ave',
'461 Main St',
'5578 Old William Penn Hwy',
'567 Livernois St',
'57 S Main St',
'5716 SE 92nd Ave',
'605 Coldsmoke Ave',
'609 Main St',
'622 Main St',
'6550 Mira Mesa Blvd',
'661 Howard St',
'7 N Saginaw St',
'700 N Pennsylvania Ave',
'812 Hamilton St',
'83 Main St',
'857 S Main St',
'8901 Yellow Brick Rd Ste B',
'918 W 1st Ave',
'930 McLaughlin Ave',
'95 Old Shoals Rd',
'9750 Indiana Pkwy']

In [56]:
dups= state_df.loc[(state_df.duplicated("Address")) & (state_df.duplicated("City"))]

In [84]:
dups_list= list(dups["Address"].unique())
dups_list= dups_list[1:len(dups_list)]

In [85]:
for i in dups_list:
    print("*"*30)
    print(i.upper())
    print(state_df.loc[state_df["Address"] == i])
    print("*"*30)

******************************
2181 LOGAN AVE
              Brewery         Address       City State ZIP Code
408  Border X Brewing  2181 Logan Ave  San Diego    CA    92113
409    Border X Logan  2181 Logan Ave  San Diego    CA    92113
******************************
******************************
18421 GOTHARD ST STE 100
                                           Brewery                   Address  \
527  Dfamle Enterprises LP (DBA Four Sons Brewing)  18421 Gothard St Ste 100   
614                              Four Sons Brewing  18421 Gothard St Ste 100   

                 City State ZIP Code  
527  Huntington Beach    CA    92648  
614  Huntington Beach    CA    92648  
******************************
******************************
6550 MIRA MESA BLVD
                    Brewery              Address       City State ZIP Code
304     Alpine Beer Company  6550 Mira Mesa Blvd  San Diego    CA    92121
642  Green Flash Brewing Co  6550 Mira Mesa Blvd  San Diego    CA    92121
**********